In [ ]:
import holoviews as hv
hv.extension('bokeh')

In [ ]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.insert(0,'../')

from IPython.core.debugger import set_trace

In [ ]:
import typyPRISM
from typyPRISM.calculate.prism.pair_correlation import pair_correlation
import numpy as np

eta = 0.45
v1 = 4.0/3.0 * np.pi * (0.5)**(3)
phi = 0.5
rho1 = phi*eta/v1
rho2 = (1-phi)*eta/v1

sys = typyPRISM.System(['A','B'],kT=1.0)
sys.domain = typyPRISM.Domain(dr=0.1,length=2048)

sys.density['A'] = rho1
sys.density['B'] = rho2


sys.potential['A','A'] = typyPRISM.potential.HardSphere(sigma=1.0)
sys.potential['B','B'] = typyPRISM.potential.HardSphere(sigma=1.0)
sys.potential['A','B'] = typyPRISM.potential.HardSphere(sigma=1.0)

sys.closure[sys.types,sys.types] = typyPRISM.closure.PercusYevick()

guess = np.zeros(sys.rank*sys.rank*sys.domain.length)
for length in [10000,2000,500]:
    sys.omega[sys.types,sys.types] = typyPRISM.omega.Gaussian(sigma=1.0,length=length)
    
    PRISM = sys.createPRISM()
    
    result = PRISM.solve(guess)
    
    guess = np.copy(PRISM.x)

guess_eps0_N500 = guess


In [ ]:
%opts Scatter Curve [width=600,height=400] Scatter (size=10,alpha=0.5)
from typyPRISM.calculate.prism.pair_correlation import pair_correlation

cv = np.array(chi_values)

hv.Scatter((cv[:,0],cv[:,1]/c50),extents=(None,None,None,None))
    

# REPULSION

In [ ]:
from typyPRISM.calculate.prism.chi import chi

sys = typyPRISM.System(['A','B'],kT=1.0)
sys.domain = typyPRISM.Domain(dr=0.1,length=2048)

sys.density['A'] = rho1
sys.density['B'] = rho2

sys.potential['A','A'] = typyPRISM.potential.HardSphere(sigma=1.0)
sys.potential['B','B'] = typyPRISM.potential.HardSphere(sigma=1.0)

sys.closure[sys.types,sys.types] = typyPRISM.closure.PercusYevick()
sys.omega[sys.types,sys.types] = typyPRISM.omega.Gaussian(sigma=1.0,length=length)


guess = np.copy(guess_eps0_N500)
for eps in [0.1,0.25,0.5,0.75,1.0]:
    print('==> eps:',eps)
    sys.potential['A','B'] = typyPRISM.potential.WeeksChandlerAndersen(sigma=1.0,epsilon=eps)

    PRISM = sys.createPRISM()
    
    result = PRISM.solve(guess)
    
    guess = np.copy(PRISM.x)
    
    c = chi(PRISM)[('A','B')][0]
    print(c,c*500)
guess_repel_N500= np.copy(guess)


In [ ]:
from typyPRISM.calculate.prism.chi import chi

sys = typyPRISM.System(['A','B'],kT=1.0)
sys.domain = typyPRISM.Domain(dr=0.1,length=1024)

sys.density['A'] = rho1
sys.density['B'] = rho2

sys.potential['A','A'] = typyPRISM.potential.HardSphere(sigma=1.0)
sys.potential['B','B'] = typyPRISM.potential.HardSphere(sigma=1.0)
sys.potential['A','B'] = typyPRISM.potential.HardCoreLennardJones(sigma=1.0,epsilon=-0.010)#+0.0005)
# sys.potential['A','B'] = typyPRISM.potential.HardCoreLennardJones(sigma=1.0,epsilon=-0.0010)#+0.0005)

sys.closure[sys.types,sys.types] = typyPRISM.closure.PercusYevick()

sys.omega[sys.types,sys.types] = typyPRISM.omega.Gaussian(sigma=1.0,length=length)

PRISM = sys.createPRISM()

result = PRISM.solve(guess_repel_N500)

guess_HCLJ_N500 = np.copy(PRISM.x)

c = chi(PRISM)[('A','B')][0]
print(c,c*500)


In [ ]:
%opts Scatter Curve [width=600,height=400]
%opts Layout [shared_axes=False]
from typyPRISM.calculate.prism.pair_correlation import pair_correlation

c = chi(PRISM)


c1 = hv.Scatter((sys.domain.k,c['A','B']),extents=(0,None,None,None))

r = sys.domain.r
U = sys.potential['A','B'].calculate(r)
c2 = hv.Curve((sys.domain.r,U),extents=(0,-0.01,2.5,0.01))

hv.Layout([c1,c2]).cols(1)

In [ ]:
from typyPRISM.calculate.prism.chi import chi

sys = typyPRISM.System(['A','B'],kT=1.0)
sys.domain = typyPRISM.Domain(dr=0.1,length=1024)

sys.density['A'] = rho1
sys.density['B'] = rho2

sys.potential['A','A'] = typyPRISM.potential.HardSphere(sigma=1.0)
sys.potential['B','B'] = typyPRISM.potential.HardSphere(sigma=1.0)
sys.potential['A','B'] = typyPRISM.potential.HardCoreLennardJones(sigma=1.0,epsilon=-0.0075)#+0.0005)

sys.closure[sys.types,sys.types] = typyPRISM.closure.PercusYevick()

sys.omega[sys.types,sys.types] = typyPRISM.omega.Gaussian(sigma=1.0,length=length)

chi_values = []
guess1 = np.copy(guess_HCLJ_N500)
for phi in np.arange(0.5,0.1,-0.05):
    sys.density['A'] = phi*eta/v1
    sys.density['B'] = (1-phi)*eta/v1
    
    print('==> Solving for...')
    print('==> phi={}'.format(phi))
    print('==> rhoA={}'.format(sys.density['A']))
    print('==> rhoB={}'.format(sys.density['B']))

    PRISM = sys.createPRISM()
    
    result = PRISM.solve(guess1)
    
    c = chi(PRISM)['A','B'][0]
    chi_values.append([phi,c])
    
    guess1 = np.copy(PRISM.x)
    
c50 = chi_values[0][1]
chi_values = chi_values[::-1]

guess1 = np.copy(guess_HCLJ_N500)
for phi in np.arange(0.5,0.9,+0.05):
    sys.density['A'] = phi*eta/v1
    sys.density['B'] = (1-phi)*eta/v1
    
    print('==> Solving for...')
    print('==> phi={}'.format(phi))
    print('==> rhoA={}'.format(sys.density['A']))
    print('==> rhoB={}'.format(sys.density['B']))

    PRISM = sys.createPRISM()
    
    result = PRISM.solve(guess1)
    
    c = chi(PRISM)['A','B'][0]
    chi_values.append([phi,c])
    
    guess1 = np.copy(PRISM.x)
chi_values = np.array(chi_values)
print(chi_values)

In [ ]:
%opts Scatter Curve [width=600,height=400] Scatter (size=10,alpha=0.5)
from typyPRISM.calculate.prism.pair_correlation import pair_correlation

cv = np.array(chi_values)

hv.Scatter((cv[:,0],cv[:,1]/c50),extents=(None,None,None,None))
    

In [ ]:
%opts Scatter Curve [width=600,height=400]
from typyPRISM.calculate.prism.pair_correlation import pair_correlation

PCF = pair_correlation(PRISM)

plots = []
for i,t,gr in PCF.itercolumn():
    c1 = hv.Curve((sys.domain.r,gr),extents=(0,None,None,None))
    plots.append(c1)
    
hv.Overlay(plots)

# ATTRACTION

In [ ]:
from typyPRISM.calculate.prism.chi import chi

sys = typyPRISM.System(['A','B'],kT=1.0)
sys.domain = typyPRISM.Domain(dr=0.1,length=1024)

sys.density['A'] = rho1
sys.density['B'] = rho2

sys.potential['A','A'] = typyPRISM.potential.HardSphere(sigma=1.0)
sys.potential['B','B'] = typyPRISM.potential.HardSphere(sigma=1.0)
sys.potential['A','B'] = typyPRISM.potential.HardCoreLennardJones(sigma=1.0,epsilon=+0.001)#+0.0005)

sys.closure[sys.types,sys.types] = typyPRISM.closure.PercusYevick()

sys.omega[sys.types,sys.types] = typyPRISM.omega.Gaussian(sigma=1.0,length=length)

chi_values = []
guess1 = np.copy(guess_eps0_N500)
for phi in np.arange(0.5,0.1,-0.05):
    sys.density['A'] = phi*eta/v1
    sys.density['B'] = (1-phi)*eta/v1
    
    print('==> Solving for...')
    print('==> phi={}'.format(phi))
    print('==> rhoA={}'.format(sys.density['A']))
    print('==> rhoB={}'.format(sys.density['B']))

    PRISM = sys.createPRISM()
    
    result = PRISM.solve(guess1)
    
    c = chi(PRISM)['A','B'][0]
    chi_values.append([phi,c])
    
    guess1 = np.copy(PRISM.x)
    
c50 = chi_values[0][1]
chi_values = chi_values[::-1]

guess1 = np.copy(guess_eps0_N500)
for phi in np.arange(0.5,0.9,+0.05):
    sys.density['A'] = phi*eta/v1
    sys.density['B'] = (1-phi)*eta/v1
    
    print('==> Solving for...')
    print('==> phi={}'.format(phi))
    print('==> rhoA={}'.format(sys.density['A']))
    print('==> rhoB={}'.format(sys.density['B']))

    PRISM = sys.createPRISM()
    
    result = PRISM.solve(guess1)
    
    c = chi(PRISM)['A','B'][0]
    chi_values.append([phi,c])
    
    guess1 = np.copy(PRISM.x)
chi_values = np.array(chi_values)
print(chi_values)

In [ ]:
c50*500

In [ ]:
%opts Scatter Curve [width=600,height=400] Scatter (size=10,alpha=0.5)
from typyPRISM.calculate.prism.pair_correlation import pair_correlation

cv = np.array(chi_values)

hv.Scatter((cv[:,0],cv[:,1]/c50),extents=(None,None,None,None))
    

# PSEUDO-ONE COMPONENT

In [ ]:
import typyPRISM
from typyPRISM.calculate.prism.pair_correlation import pair_correlation
import numpy as np

eta = 0.45
v1 = 4.0/3.0 * np.pi * (0.5)**(3)
phi = 0.5
rho1 = phi*eta/v1
rho2 = (1-phi)*eta/v1

sys = typyPRISM.System(['A','B'],kT=1.0)
sys.domain = typyPRISM.Domain(dr=0.1,length=1024)

sys.density['A'] = rho1 + rho2


sys.potential['A','A'] = typyPRISM.potential.HardSphere(sigma=1.0)
sys.potential['B','B'] = typyPRISM.potential.HardSphere(sigma=1.0)
sys.potential['A','B'] = typyPRISM.potential.HardSphere(sigma=1.0)

sys.closure[sys.types,sys.types] = typyPRISM.closure.PercusYevick()

guess = np.zeros(sys.rank*sys.rank*sys.domain.length)
for length in [10000,2000,500]:
    sys.omega[sys.types,sys.types] = typyPRISM.omega.Gaussian(sigma=1.0,length=length)
    
    PRISM = sys.createPRISM()
    
    result = PRISM.solve(guess)
    
    guess = np.copy(PRISM.x)

guess_eps0_N500 = guess